In [1]:
%config InlineBackend.figure_format = "retina"
%config InlineBackend.figure_formats = ["pdf", "svg"]
from preamble import *

## 1. 전처리 : Pandas 데이터로 변경 

In [2]:
## pd.read_csv("data/ratings_train.txt")  ParserError: Error tokenizing data. C error: Expected 1 fields in line 21, saw 2 에러 발생, 해석 에러!!!!
df_train = pd.read_csv("data/ratings_train.txt",
                        delimiter="\t",
                        keep_default_na=False) ## 구분자 값을 "\t"값으로 지정 및 na값을 제거
# df_train.head() ## df_train의 헤드 출력
df_test = pd.read_csv("data/ratings_test.txt",
                        delimiter="\t",
                        keep_default_na=False) ## 구분자 값을 "\t"값으로 지정 및 na값을 제거
# df_test.head()



2. 데이터 나누기(훈련, 테스트)

In [3]:
text_train, y_train = df_train["document"].values, df_train["label"].values
text_test,y_test = df_test["document"].values, df_test["label"].values
print(len(text_train), len(text_test))

150000 50000


### 3. 특징추출 : 한국어 데이터는 무조건 해야함, 한국어를 숫자로 변환해 줘야 한다.  -> 문자를 숫자로 변환해야 함. (Token)
### 임베딩(Embedding) : 텍스트, 이미지, 카테고리 등의 데이터를 수치화된 벡터 형태로 변환하는 기법
### 토큰을 분류해주는 라이브러리가 필요하다.

## 자바가 설치되어 있어야 한다.
## java --version 자바 버전 확인
## pip install konlpy 라이브러리 설치

In [4]:
from sklearn.ensemble import RandomForestClassifier # ValueError: could not convert string to float: '아 더빙.. 진짜 짜증나네요 목소리' ## 한국어 문자는 float으로 바꾸는 것은 없다.
rf = RandomForestClassifier()
rf.fit(text_train, y_train)
rf.score(text_test, y_test)

ValueError: could not convert string to float: '아 더빙.. 진짜 짜증나네요 목소리'

### 3. 특징추출 ### Okt 사용
## 이미지 => 전처리
## 텍스트 => 임베딩 과정
### 말뭉치(코퍼스, Corpus) : 자연어 처리(NLP)와 언어학에서 분석 및 연구를 위해 수집된 텍스트 데이터의 집합

In [6]:
from konlpy.tag import Okt # 언어 맥락을 잘 알고 하는것이 매우 도움된다.
okt = Okt()
ko_text = "한국어 분석을 시작해봐요!"
morphs = okt.morphs(ko_text) # morphs()는 문장을 형태소(의미를 가진 최소 단위)로 분리
pos = okt.pos(ko_text)
nouns = okt.nouns(ko_text)
print(f"형태소 : {morphs}")
print(f"품사 : {pos}")
print(f"nouns : {nouns}")

형태소 : ['한국어', '분석', '을', '시작', '해봐요', '!']
품사 : [('한국어', 'Noun'), ('분석', 'Noun'), ('을', 'Josa'), ('시작', 'Noun'), ('해봐요', 'Verb'), ('!', 'Punctuation')]
nouns : ['한국어', '분석', '시작']


### B-o-W (Bag of Words, 단어 가방)
- 자연어 처리(NLP)에서 텍스트 데이터를 수치화하는 가장 기본적인 특징 추출(Feature Extraction) 기법
- 텍스트를 단어의 등장 빈도로 표현하여 기계 학습 모델이 텍스트를 처리할 수 있도록 변환

In [10]:
okt = Okt()
def okt_tokenizer(text) :
  return okt.morphs(text)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer # CountVectorizer 빈도수
vectorizer = CountVectorizer(tokenizer = okt_tokenizer,
                             token_pattern=None,
                             min_df = 5, # 최소 5번은 나와야 함
                             max_df = 0.9) # 90% 이상
X_train = vectorizer.fit_transform(text_train)
X_test = vectorizer.fit_transform(text_test)


In [21]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer # CountVectorizer 빈도수
vectorizer = TfidfVectorizer(tokenizer = okt_tokenizer,
                             token_pattern=None,
                             min_df = 5, # 최소 5번은 나와야 함
                             max_df = 0.9) # 90% 이상
X_train = vectorizer.fit_transform(text_train)
X_test = vectorizer.fit_transform(text_test)


In [25]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier()
tree.fit(X_train, y_train)
tree.score(X_train,y_train)

0.9947933333333333

In [24]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)
lr.score(X_train, y_train)

0.6427048972941163